In [3]:
# Import necessary modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
from pathlib import Path
from datetime import datetime, timedelta
import json
import sys
import math
from geopy.distance import distance
from matplotlib.colors import to_rgba
from collections import defaultdict
import simplekml
import matplotlib.cm as cm
from concurrent.futures import ThreadPoolExecutor , as_completed


In [9]:

# Establish directory locations

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/kml')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')

In [10]:
# Plotting preferences

# Global
min_lon = -35 #-35
max_lon = -5 #-5
min_lat = 55 #55
max_lat = 66 #66

additional_suffix = '_TEST'

# Float colors
# For additional floats, add the name (as it appears in the 'Float_positions.csv') and color you'd like (as a hexcode)
float_colors = {
        '4903532':  '#B4184C',
        'navis102': '#F5A300',
        '1902637':  '#0000E0',
        'navis101': '#FBFF1F'}

# CHLA
skip_chla            = False
CHLA_color           = 'YlGnBu_r'
CHLA_plot_as_log     = True # Default True
CHLA_plot_lim_max    = 10   # Default 10
CHLA_plot_lim_min    = 0.1  # Default 0.1

In [11]:
### Plotting most recent data from CMEMS
## List most recent files

# List to store matching file names
matching_files = []

# Loop through the files in the directory
for filename in os.listdir(satellite_dir):
    if filename.endswith('.nc'):
        matching_files.append(filename)

print(matching_files)

## List variables from these files
# Dictionary to store files by their base names (excluding '_a_' or '_b_') and key variables
file_dict = {}

for file in matching_files:
    # Identify the base name by removing '_a' or '_b' if present
    base_name = file.replace('_a', '_').replace('_b', '_')
    if base_name not in file_dict:
        file_dict[base_name] = {}
    filepath = os.path.join(satellite_dir, file)
    ds = xr.open_dataset(filepath)
    variables = list(ds.data_vars.keys())

    print(file)
    print(variables)

    key_variables = [var for var in variables if 'longitude' not in var.lower() and 'latitude' not in var.lower()]

    if key_variables:
        for key_variable in key_variables:
            if file not in file_dict[base_name]:
                file_dict[base_name][file] = {}
            file_dict[base_name][file][key_variable] = {
                'variable': key_variable,
            }
    else:
        print(f"No suitable key variable found in file: {file}")

    ds.close()

# Initialize var_dict with categories as keys and empty dictionaries as values
var_dict = {'chl': {}, 'adt': {}}

for base_name, files in file_dict.items():
    for file, variables in files.items():
        for key_variable, details in variables.items():
            data_var = details['variable']
            var_name_lower = data_var.lower()
            if 'chl' in var_name_lower and var_name_lower == 'chl':
                var_dict['chl'][file] = data_var
            elif 'adt' in var_name_lower:
                var_dict['adt'][file] = data_var

#first_line_path  = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_J_1000.shp'
#second_line_path = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_I_2000.shp'

#gdf_1000 = gpd.read_file(first_line_path)
#gdf_2000 = gpd.read_file(second_line_path)

['CMEMS_cmems_mod_glo_bgc-car_anfc_0.25deg_P1D-m_April-May2024.nc', 'CMEMS_cmems_obs-oc_atl_bgc-optics_nrt_l3-multi-1km_P1D_April-May2024.nc', 'CMEMS_cmems_obs-oc_atl_bgc-plankton_nrt_l3-multi-1km_P1D_April-May2024.nc', 'CMEMS_cmems_obs-sl_eur_phy-ssh_nrt_allsat-l4-duacs-0.125deg_P1D_April-May2024.nc', 'CMEMS_cmems_obs_mob_glo_phy-cur_nrt_0.25deg_P1D-m_April-May2024.nc']
CMEMS_cmems_mod_glo_bgc-car_anfc_0.25deg_P1D-m_April-May2024.nc
['dissic', 'ph', 'talk']
CMEMS_cmems_obs-oc_atl_bgc-optics_nrt_l3-multi-1km_P1D_April-May2024.nc
['BBP', 'BBP_uncertainty', 'CDM', 'CDM_uncertainty', 'flags']
CMEMS_cmems_obs-oc_atl_bgc-plankton_nrt_l3-multi-1km_P1D_April-May2024.nc
['CHL', 'CHL_uncertainty', 'DIATO', 'DIATO_uncertainty', 'DINO', 'DINO_uncertainty', 'GREEN', 'GREEN_uncertainty', 'HAPTO', 'HAPTO_uncertainty', 'MICRO', 'MICRO_uncertainty', 'NANO', 'NANO_uncertainty', 'PICO', 'PICO_uncertainty', 'PROCHLO', 'PROCHLO_uncertainty', 'PROKAR', 'PROKAR_uncertainty', 'flags']
CMEMS_cmems_obs-sl_eur_

In [6]:

# Define the colormap and normalization
color = CHLA_color
xx_plot_min = CHLA_plot_lim_min
xx_plot_max = CHLA_plot_lim_max
log_scaling = CHLA_plot_as_log

cmap  = plt.get_cmap(color)
norm  = colors.LogNorm(vmin=xx_plot_min, vmax=xx_plot_max) if log_scaling else colors.Normalize(vmin=xx_plot_min, vmax=xx_plot_max)

csv_file = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\Plotting_tools\\shared_data\\rt_positions.csv'  # Update this path
df = pd.read_csv(csv_file)
df['date'] = pd.to_datetime(df['date'])

# Define unique symbols for different object types
object_symbols = {
    'glider': 'o',
    'Float': 's',
    'Ship'  : '^'
    # Add more object types and their symbols here
}

# Define colors for different objects
object_colors = {
    'unit_345': '#53599A',  # Example color
    'unit_397': '#7A7FB8',
    'unit_398': '#AFB3D4',
    'unit_405': '#32365D',
    'lovuse026d': '#F7A072',  # Example color
    'lovuse031c': '#F47E3E',  # Example color
    'lovuse032c': '#D4520C',  # Example color
    'Discovery' : '#70163C'
    # Add more object names and their colors here
}

def color_to_kml_format(color):
    return simplekml.Color.rgb(
        int(color[0] * 255),
        int(color[1] * 255),
        int(color[2] * 255),
        int(color[3] * 255)
    )

#kml_colors = [color_to_kml_format(color) for color in colors_im]

def create_kml_polygon(path, kml_color, date_of_plot_filter):
    coords = path.vertices
    lon_lat_coords = [(lon, lat) for lon, lat in zip(coords[:, 0], coords[:, 1])]
    
    pol = folder.newpolygon()
    pol.outerboundaryis = lon_lat_coords
    pol.style.polystyle.color = kml_color
    pol.style.linestyle.color = simplekml.Color.changealphaint(0, kml_color)  # Set border to transparent
    pol.timestamp.when = date_of_plot_filter

NameError: name 'colors_im' is not defined

In [ ]:
# KML File Loop

for var_category, files in var_dict.items():

    if var_category == 'chl':

        for day_offset in [-1]:  # Plot the last two days # Change this silly bit of code to be something more readable
            
            if len(files) > 1:
                data_vars = []
                for file, data_var in files.items():
                    cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                    data_vars.append(cur_data_var[data_var].isel(time=day_offset))
                aligned_data_vars = xr.align(*data_vars, join='outer')
                combined_data_var = aligned_data_vars[0]
                for var in aligned_data_vars[1:]:
                    combined_data_var = combined_data_var.combine_first(var)
                combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
                date_of_plot = str(combined_data_var['time'].data[0])[0:10]
            elif len(files) == 1:
                file, data_var = next(iter(files.items()))
                cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                date_of_plot = str(cur_data_var['time'].data[day_offset])[0:10]
                combined_data_var_mean = cur_data_var[data_var].isel(time=day_offset).data
            else:
                continue
    
        # Units for colorbar
        xx_plot_units      = cur_data_var[data_var].attrs.get('units')
        xx_plot_cbar_label = 'Sea surface velocity (' + xx_plot_units + 's$^{-1}$)'

        # Clip the data within the specified range
        data_to_plot_1day = np.clip(combined_data_var_mean, xx_plot_min, xx_plot_max)
        
        # If the first dimension of the data is 1, remove it
        if data_to_plot_1day.shape[0] == 1:
           data_to_plot_1day = data_to_plot_1day[0]
        
        # Extract longitudes and latitudes
        longitudes = cur_data_var['longitude'].data
        latitudes = cur_data_var['latitude'].data
        
        # Extracting latest lead float position

        date_of_plot_filter = datetime.strptime(date_of_plot, '%Y-%m-%d')
        df_filtered = df[df['date'] <= date_of_plot_filter]

        legend_handles = []

        # Process each unique object type
        for obj_type, symbol in object_symbols.items():

            if obj_type == 'Float':

                # Filter data for the current object type
                type_data = df_filtered[df_filtered['platform_type'] == obj_type]
                unique_names = type_data['platform_id'].unique()

                obj_data = type_data.sort_values(by='date', ascending=False)

        # Define the bounds of the 2-degree square around the center
        lon_min = obj_data['lon'].iloc[0] - 2
        lon_max = obj_data['lon'].iloc[0] + 2
        lat_min = obj_data['lat'].iloc[0] - 2
        lat_max = obj_data['lat'].iloc[0] + 2

        # Find the indices of the longitudes and latitudes within the bounds
        lon_indices = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[0]
        lat_indices = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[0]

        # Extract the subset of data within the bounds
        subsampled_longitudes = longitudes[lon_indices]
        subsampled_latitudes = latitudes[lat_indices]
        subsampled_data = data_to_plot_1day[np.ix_(lat_indices, lon_indices)]

        # Create the figure and axis
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(subsampled_longitudes, 
                           subsampled_latitudes, 
                           subsampled_data, 
                           cmap=cmap, 
                           norm=norm, 
                           transform=ccrs.PlateCarree())
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax, label=xx_plot_cbar_label)
        
        # Create a KML object
        kml = simplekml.Kml()
        
        # Get the vertices from the pcolormesh plot
        paths = im.get_paths()
        
        # Get the array data used to color the polygons
        data_values = im.get_array()
        
        # Convert data values to RGBA colors using the colormap and norm
        normed_data = norm(data_values)
        colors_im = cmap(normed_data)
        colors_im = colors_im.reshape((-1, 4))
        
        # Save the colorbar image
        fig, cax = plt.subplots(figsize=(2, 6))
        fig.subplots_adjust(right=0.5)
        cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
        cbar.set_label(xx_plot_cbar_label)
        colorbar_filename = "colorbar.png"
        fig.savefig(colorbar_filename, bbox_inches='tight')
        
        # Create a KML object
        kml = simplekml.Kml()
        
        # Create a Folder to hold the colorbar and the data
        folder = kml.newfolder(name="Data with Colorbar")
        
        # Add the pcolormesh to the KML
        paths = im.get_paths()
        data_values = im.get_array()
        normed_data = norm(data_values)
        colors_im = cmap(normed_data)
        colors_im = colors_im.reshape((-1, 4))
        
        with ThreadPoolExecutor() as executor:
            futures = [
                executor.submit(create_kml_polygon, path, kml_color, date_of_plot_filter)
                for path, kml_color in zip(paths, kml_colors)
            ]
            
            for _ in tqdm(as_completed(futures), total=len(futures)):
                pass
        
        # Add the colorbar as a ScreenOverlay
        screen = folder.newscreenoverlay(name='Colorbar')
        screen.icon.href = colorbar_filename
        screen.overlayxy = simplekml.OverlayXY(x=0, y=0, xunits=simplekml.Units.fraction, yunits=simplekml.Units.fraction)
        screen.screenxy = simplekml.ScreenXY(x=0.05, y=0.95, xunits=simplekml.Units.fraction, yunits=simplekml.Units.fraction)
        screen.size.x = 0.1
        screen.size.y = 0.4
        screen.size.xunits = simplekml.Units.fraction
        screen.size.yunits = simplekml.Units.fraction
        
        # Save the KML file
        kml.save(f"{output_dir}_CHLA_{date_of_plot}.kml")


In [12]:

for var_category, files in var_dict.items():
    if var_category == 'adt':
        for day_offset in [-1]:  # Plot the last day
            if len(files) > 1:
                data_vars = []
                for file, data_var in files.items():
                    cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                    data_vars.append(cur_data_var[data_var].isel(time=day_offset))
                aligned_data_vars = xr.align(*data_vars, join='outer')
                combined_data_var = aligned_data_vars[0]
                for var in aligned_data_vars[1:]:
                    combined_data_var = combined_data_var.combine_first(var)
                combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
                date_of_plot = str(combined_data_var['time'].data[0])[0:10]
            elif len(files) == 1:
                file, data_var = next(iter(files.items()))
                cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                date_of_plot = str(cur_data_var['time'].data[day_offset])[0:10]
                combined_data_var_mean = cur_data_var[data_var].isel(time=day_offset).data
            else:
                continue

            # Units for colorbar
            xx_plot_units = cur_data_var[data_var].attrs.get('units')
            xx_plot_cbar_label = 'Sea surface velocity (' + xx_plot_units + 's$^{-1}$)'

            # Clip the data within the specified range
            data_to_plot_1day = np.clip(combined_data_var_mean, xx_plot_min, xx_plot_max)

            # If the first dimension of the data is 1, remove it
            if data_to_plot_1day.shape[0] == 1:
                data_to_plot_1day = data_to_plot_1day[0]

            # Extract longitudes and latitudes
            longitudes = cur_data_var['longitude'].data
            latitudes = cur_data_var['latitude'].data
            ugos = cur_data_var['ugos'].isel(time=day_offset).data
            vgos = cur_data_var['vgos'].isel(time=day_offset).data

            # Extracting latest lead float position
            date_of_plot_filter = datetime.strptime(date_of_plot, '%Y-%m-%d')
            df_filtered = df[df['date'] <= date_of_plot_filter]

            legend_handles = []

            # Process each unique object type
            for obj_type, symbol in object_symbols.items():
                if obj_type == 'Float':
                    type_data = df_filtered[df_filtered['platform_type'] == obj_type]
                    unique_names = type_data['platform_id'].unique()
                    obj_data = type_data.sort_values(by='date', ascending=False)

            # Define the bounds of the 2-degree square around the center
            lon_min = obj_data['lon'].iloc[0] - 2
            lon_max = obj_data['lon'].iloc[0] + 2
            lat_min = obj_data['lat'].iloc[0] - 2
            lat_max = obj_data['lat'].iloc[0] + 2

            # Find the indices of the longitudes and latitudes within the bounds
            lon_indices = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[0]
            lat_indices = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[0]

            # Extract the subset of data within the bounds
            subsampled_longitudes = longitudes[lon_indices]
            subsampled_latitudes = latitudes[lat_indices]
            subsampled_ugos = ugos[np.ix_(lat_indices, lon_indices)]
            subsampled_vgos = vgos[np.ix_(lat_indices, lon_indices)]

            fig = plt.figure(figsize=(15, 15))
            ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
            ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

            # Extract the lon and lat from the dataset only once
            x = subsampled_longitudes
            y = subsampled_latitudes

            # From the U and V vector compute the speed, we use it as our colour map
            u = subsampled_ugos
            v = subsampled_vgos

            time_step = 6 * 3600  # Convert 6 hours to seconds

            U_km_per_6hr = u * time_step / 1000  # Convert from m/s to km/6hr
            V_km_per_6hr = v * time_step / 1000  # Convert from m/s to km/6hr
            speed = np.sqrt(u**2 + v**2)

            # Plot the current vectors field and the coastline
            im = ax.quiver(x, y, U_km_per_6hr, V_km_per_6hr, speed, angles='xy', scale_units='xy', cmap='viridis', transform=ccrs.PlateCarree(), width=0.002)
            png_filename = 'quiver_plot.png'
            plt.savefig(png_filename, bbox_inches='tight', pad_inches=0)
            plt.close()

            # Define the bounding box for the overlay (corresponding to the plot extent)
            north = lat_max
            south = lat_min
            east  = lon_max
            west  = lon_min
            
            # Create a KML object
            kml = simplekml.Kml()
            
            # Add the image overlay
            ground = kml.newgroundoverlay(name='Quiver Plot')
            ground.icon.href = png_filename
            ground.latlonbox.north = north
            ground.latlonbox.south = south
            ground.latlonbox.east = east
            ground.latlonbox.west = west
            
            # Save the KML file
            kml_filename = 'quiver_plot.kml'
            kml.save(kml_filename)


